In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
import glob
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import cv2
from imblearn.over_sampling import SMOTE
import pywt

# Label extraction

In [2]:
df=pd.read_csv("oasis_longitudinal_demographics.csv")

In [3]:
df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [4]:
labels=df['Group']

In [5]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [6]:
label_to_id_dict

{'Converted': 0, 'Demented': 1, 'Nondemented': 2}

In [7]:
label_ids = np.array([label_to_id_dict[x] for x in labels])

In [8]:
label_ids

array([2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 0,
       0, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1,

# Image extraction

In [9]:
files=glob.glob("OAS2_RAW_PART1/*")

In [10]:
full_features=[]
segmented_features=[]
for file_path in files:
    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')
    img = nib.load(filename)
    image_data = img.get_fdata()
    image=ndimage.rotate(image_data[:, :, 64,0],90)
    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)
    full_features.append(thresh3)
    segmented_features.append(thresh3[70:150,70:170])

In [11]:
len(full_features),len(segmented_features)

(209, 209)

In [12]:
files=glob.glob("OAS2_RAW_PART2/*")

In [13]:
for file_path in files:
    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')
    img = nib.load(filename)
    image_data = img.get_fdata()
    image=ndimage.rotate(image_data[:, :, 64,0],90)
    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)
    full_features.append(thresh3)
    segmented_features.append(thresh3[70:150,70:170])

In [14]:
len(full_features),len(segmented_features)

(373, 373)

In [15]:
full_features=np.array(full_features)
segmented_features=np.array(segmented_features)

# Train test

In [16]:
X_full_train, X_full_test, y_train, y_test = train_test_split(full_features,label_ids, test_size = 0.20)  
X_seg_train, X_seg_test, y_train, y_test = train_test_split(segmented_features,label_ids, test_size = 0.20) 

In [17]:
X_full_train[0].shape,X_seg_train[0].shape

((256, 256), (80, 100))

In [18]:
#Normalize color values to between 0 and 1
X_full_train = X_full_train/255
X_full_test = X_full_test/255

X_seg_train = X_seg_train/255
X_seg_test = X_seg_test/255

X_full_flat_train = X_full_train.reshape(X_full_train.shape[0], 256*256*1)
X_full_flat_test = X_full_test.reshape(X_full_test.shape[0], 256*256*1)

X_seg_flat_train = X_seg_train.reshape(X_seg_train.shape[0], 8000)
X_seg_flat_test = X_seg_test.reshape(X_seg_test.shape[0], 8000)


In [20]:
print(X_seg_flat_train)

[[ 2.55686275  1.44705882  2.23529412 ...  1.30980392  0.
   0.74117647]
 [ 2.25098039  5.41960784  6.23137255 ...  3.87843137  4.45882353
   4.29803922]
 [ 1.54117647  2.87843137  2.88627451 ...  3.34509804  4.53333333
   4.33333333]
 ...
 [ 4.35294118  3.71372549  4.95686275 ...  2.89411765  3.70588235
   3.08627451]
 [14.81176471 13.71764706 11.76078431 ...  2.81568627  2.96078431
   3.09803922]
 [ 1.70980392  1.43921569  1.98039216 ...  2.91372549  2.10980392
   2.56078431]]


In [19]:
#X_3d_train = X_train.reshape(X_train.shape[0], 64,64,1)
#X_3d_test = X_test.reshape(X_test.shape[0], 64,64,1)

In [20]:
#X_3d_train.shape

In [21]:
#y_train=label_ids[:298]
#y_test=label_ids[298:373]

# Multilayer model with oversampling

In [22]:
sm=SMOTE(random_state=42)

train_full_data, train_full_labels = sm.fit_resample(X_full_flat_train, y_train)
train_seg_data, train_seg_labels = sm.fit_resample(X_seg_flat_train, y_train)

In [23]:
train_full_data.shape,train_seg_data.shape

((435, 65536), (435, 8000))

# Ada Boost

### Basic model Full features

In [29]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(X_full_flat_train, y_train)
y_pred3 = model3.predict(X_full_flat_test)

In [30]:
accuracy_score(y_test, y_pred3)

0.56

### Grid search cv full features

In [95]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [96]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [97]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.333 total time=  47.2s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.383 total time=  43.6s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.367 total time=  44.0s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.475 total time=  48.6s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.407 total time=  44.2s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.317 total time= 1.0min
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.217 total time=  56.8s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.367 total time= 1.1min
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.441 total time= 1.0min
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.37

C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30;, score=nan total time=  27.8s


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30;, score=nan total time=  25.0s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50;, score=nan total time=  22.1s


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50;, score=nan total time=  17.9s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.450 total time=  41.8s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.433 total time=  41.8s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.450 total time=  45.7s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.373 total time=  43.6s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.390 total time=  47.0s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.467 total time=  57.4s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.467 total time= 1.1min
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.400 total time= 1.0min
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.441 total time=  57.7s
[CV 5/5] END algorithm=S

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.39293785 0.34271186 0.43322034        nan        nan        nan
        nan        nan        nan 0.41920904 0.43954802 0.36920904
 0.40276836 0.41282486 0.47322034 0.34881356 0.45011299 0.40638418]
  category=UserWarning


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [98]:
grid.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 4, 'n_estimators': 50}

In [99]:
model1=grid.best_estimator_

In [100]:
y_pred1= model1.predict(X_full_flat_test)

In [101]:
accuracy_score(y_test, y_pred1)

0.4

### Basic model full features oversampling

In [65]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(train_full_data, train_full_labels)
y_pred3 = model3.predict(X_full_flat_test)

In [66]:
accuracy_score(y_test, y_pred3)

0.52

### Grid search cv model full features oversampling

In [24]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [25]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [26]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  38.4s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  39.1s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  38.7s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  37.7s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  40.4s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  52.0s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  52.5s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  53.0s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  53.4s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  52.5s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=50; total time= 1.1min
[CV 2/5] END algorithm=SAM

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=  24.8s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=  22.7s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=  19.9s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=  17.7s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  39.1s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  37.7s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  39.9s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  40.3s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  40.0s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  52.0s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  50.1s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  52.2s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  52.6s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  54.4s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; t

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.36879552 0.46843137 0.48005602        nan        nan        nan
        nan        nan        nan 0.43739496 0.4372549  0.43501401
 0.37582633 0.47053221 0.29540616 0.42588235 0.36383754 0.37128852]
  warnings.warn(


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [27]:
grid.best_params_

{'algorithm': 'SAMME', 'learning_rate': 3, 'n_estimators': 50}

In [28]:
model1=grid.best_estimator_

In [29]:
y_pred1= model1.predict(X_full_flat_test)

In [30]:
accuracy_score(y_test, y_pred1)

0.24

### Basic model segmented features

In [37]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(X_seg_flat_train, y_train)
y_pred3 = model3.predict(X_seg_flat_test)

In [38]:
accuracy_score(y_test, y_pred3)

0.5866666666666667

### Grid search cv segmented features

In [55]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [56]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [57]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.483 total time=   6.3s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.450 total time=   6.1s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.417 total time=   6.2s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.288 total time=   6.2s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.576 total time=   6.3s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.450 total time=   8.5s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.400 total time=   9.0s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.467 total time=   9.5s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.492 total time=   9.4s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.54

C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30;, score=nan total time=   3.4s


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 3/5] END algorithm=SAMME, learning_rate=4, n_estimators=30;, score=nan total time=   3.6s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50;, score=nan total time=   2.1s


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50;, score=nan total time=   2.4s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.400 total time=   6.0s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.467 total time=   6.4s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.550 total time=   6.0s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.339 total time=   6.4s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.525 total time=   6.1s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.467 total time=   8.2s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.283 total time=   9.1s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.433 total time=  10.3s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.373 total time=   9.8s
[CV 5/5] END algorithm=S

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.44288136 0.47011299 0.50367232        nan        nan        nan
        nan        nan        nan 0.45621469 0.38242938 0.49977401
 0.41971751 0.41632768 0.45265537 0.29525424 0.41322034 0.39627119]
  category=UserWarning


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [58]:
grid.best_params_

{'algorithm': 'SAMME', 'learning_rate': 3, 'n_estimators': 50}

In [59]:
model1=grid.best_estimator_

In [60]:
y_pred1= model1.predict(X_seg_flat_test)

In [61]:
accuracy_score(y_test, y_pred1)

0.4

### Basic model segmented features oversampling

In [67]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(train_seg_data, train_seg_labels)
y_pred3 = model3.predict(X_seg_flat_test)

In [68]:
accuracy_score(y_test, y_pred3)

0.52

### Grid search cv segmented features oversampling

In [69]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [70]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [71]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.374 total time=  10.8s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.352 total time=  10.5s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.352 total time=  10.7s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.189 total time=  10.4s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30;, score=0.422 total time=  10.1s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.418 total time=  13.3s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.549 total time=  13.3s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.538 total time=  13.3s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.433 total time=  13.3s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40;, score=0.46

C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30;, score=nan total time=   5.4s


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30;, score=nan total time=   7.3s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50;, score=nan total time=   5.8s


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:610: RuntimeWarning: overflow encountered in exp
  (sample_weight > 0))
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 134, in

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50;, score=nan total time=  10.8s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.451 total time=  10.4s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.308 total time=  11.0s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.363 total time=  11.1s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.533 total time=  11.2s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30;, score=0.322 total time=  13.7s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.451 total time=  19.5s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.451 total time=  17.7s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.363 total time=  18.3s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40;, score=0.422 total time=  15.4s
[CV 5/5] END algorithm=S

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.33760684 0.4810989  0.4173138         nan        nan        nan
        nan        nan        nan 0.39528694 0.40385836 0.44358974
 0.33140415 0.33985348 0.3352381  0.30251526 0.3578022  0.33775336]
  category=UserWarning


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [72]:
grid.best_params_

{'algorithm': 'SAMME', 'learning_rate': 3, 'n_estimators': 40}

In [73]:
model1=grid.best_estimator_

In [74]:
y_pred1= model1.predict(X_seg_flat_test)

In [75]:
accuracy_score(y_test, y_pred1)

0.4

# Gradient boosting

### Basic model full features

In [42]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(X_full_flat_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [43]:
y_pred1= clf.predict(X_full_flat_test)

In [44]:
accuracy_score(y_test, y_pred1)

0.4533333333333333

### Basic model full features oversampling

In [80]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(train_full_data, train_full_labels)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [81]:
y_pred1= clf.predict(X_full_flat_test)

In [82]:
accuracy_score(y_test, y_pred1)

0.52

### Basic model segmented features

In [45]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(X_seg_flat_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [46]:
y_pred1= clf.predict(X_seg_flat_test)

In [47]:
accuracy_score(y_test, y_pred1)

0.56

### Basic model seg features oversampling

In [76]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(train_seg_data, train_seg_labels)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [78]:
y_pred1= clf.predict(X_seg_flat_test)

In [79]:
accuracy_score(y_test, y_pred1)

0.5466666666666666

# Voting classifier

In [85]:
model1=LogisticRegression(random_state=1)
model2=DecisionTreeClassifier(random_state=1)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

### Basic model full features

In [86]:
model.fit(X_full_flat_train, y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [87]:
model.score(X_full_flat_test,y_test)

0.41333333333333333

### Basic model full features oversampling

In [88]:
model.fit(train_full_data, train_full_labels)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [89]:
model.score(X_full_flat_test,y_test)

0.4266666666666667

### Basic model segmented features

In [90]:
model.fit(X_seg_flat_train, y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [92]:
model.score(X_seg_flat_test,y_test)

0.3333333333333333

### Basic model full features oversampling

In [93]:
model.fit(train_seg_data, train_seg_labels)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [94]:
model.score(X_seg_flat_test,y_test)

0.44

# Random Forests

### Basic model full features

In [25]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [26]:
clf.fit(X_full_flat_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [27]:
ypred=clf.predict(X_full_flat_test)

In [28]:
accuracy_score(y_test, ypred)

0.52

### Grid search cv full features

In [29]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [30]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [31]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.9s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.9s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.9s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.9s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.6s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.6s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.3s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.1s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.9s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.4s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.4s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.4s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.4s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.3s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.3s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.4s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.5s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.3s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.4s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.4s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.6s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.6s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.6s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.6s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [32]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 'sqrt',
 'n_estimators': 200}

In [33]:
y_pred1=grid.predict(X_full_flat_test)

In [34]:
accuracy_score(y_test, y_pred1)

0.5066666666666667

### Basic model oversampling full features

In [35]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [36]:
clf.fit(train_full_data, train_full_labels)

RandomForestClassifier(max_depth=2, random_state=0)

In [37]:
ypred=clf.predict(X_full_flat_test)

In [38]:
accuracy_score(y_test, ypred)

0.5333333333333333

### Grid search cv full features oversampling

In [47]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [48]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [49]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.8s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.7s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.7s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.7s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.7s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   1.4s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   1.4s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   1.4s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   1.5s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   1.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   1.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   2.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   2.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   2.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   2.1s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   2.1s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.2s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.2s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.1s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   4.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   3.9s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   3.9s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   3.8s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   5.2s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   5.1s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   5.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   5.2s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   5.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   5.6s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   5.5s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   7.5s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   7.4s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   7.4s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   7.5s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   7.3s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.1s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   2.9s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   2.9s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   2.9s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   2.8s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [50]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'log2',
 'n_estimators': 300}

In [51]:
y_pred1=grid.predict(X_full_flat_test)

In [52]:
accuracy_score(y_test, y_pred1)

0.5333333333333333

### Basic model segmented features

In [39]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [40]:
clf.fit(X_seg_flat_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [41]:
ypred=clf.predict(X_seg_flat_test)

In [42]:
accuracy_score(y_test, ypred)

0.5466666666666666

### Grid search cv segmented features

In [53]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [54]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [55]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.8s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.8s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.8s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.4s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.5s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.5s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.9s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.1s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [56]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [57]:
y_pred1=grid.predict(X_seg_flat_test)

In [58]:
accuracy_score(y_test, y_pred1)

0.56

### Basic model oversampling segmented features

In [43]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [44]:
clf.fit(train_seg_data, train_seg_labels)

RandomForestClassifier(max_depth=2, random_state=0)

In [45]:
ypred=clf.predict(X_seg_flat_test)

In [46]:
accuracy_score(y_test, ypred)

0.5333333333333333

### Grid search cv full features oversampling

In [59]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [60]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [61]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.6s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.6s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.2s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.2s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.2s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.6s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.6s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.6s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.6s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.2s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.2s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.2s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.2s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.2s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.3s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.3s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   3.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   3.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   3.1s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   3.1s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   3.1s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.8s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.8s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.8s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.1s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.1s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.1s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.1s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [62]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 400}

In [63]:
y_pred1=grid.predict(X_seg_flat_test)

In [64]:
accuracy_score(y_test, y_pred1)

0.5333333333333333

# Extra trees classifier

### Basic model full features

In [65]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(X_full_flat_train, y_train)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [66]:
ypred=extra_tree_forest.predict(X_full_flat_test)

In [67]:
accuracy_score(ypred,y_test)

0.4533333333333333

### Grid search cv full features

In [68]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [69]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [70]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3

[CV 3/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4

[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5

[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] E

[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] E

[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] E

[CV 3/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s


GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [71]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'sqrt',
 'n_estimators': 9}

In [72]:
y_pred1=grid.predict(X_full_flat_test)

In [73]:
accuracy_score(y_test, y_pred1)

0.56

### Basic model full features Oversampling

In [24]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(train_full_data, train_full_labels)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [25]:
ypred=extra_tree_forest.predict(X_full_flat_test)

In [26]:
accuracy_score(ypred,y_test)

0.4

### Grid search cv full features

In [27]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [28]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [29]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=3

[CV 2/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=4

[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=5

[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.1s
[CV 3/5] E

[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.1s
[CV 2/5] E

[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.1s
[CV 5/5] E

[CV 3/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s


GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [30]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 7}

In [31]:
y_pred1=grid.predict(X_full_flat_test)

In [32]:
accuracy_score(y_test, y_pred1)

0.48

### Basic model segmented features

In [33]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(X_seg_flat_train, y_train)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [34]:
ypred=extra_tree_forest.predict(X_seg_flat_test)

In [35]:
accuracy_score(ypred,y_test)

0.49333333333333335

### Grid search cv full features

In [36]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [37]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [38]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3

[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4

[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5

[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] E

[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] E

[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 2/5] E

GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [39]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 8}

In [40]:
y_pred1=grid.predict(X_seg_flat_test)

In [41]:
accuracy_score(y_test, y_pred1)

0.6533333333333333

### Basic model full features Oversampling

In [42]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(train_seg_data, train_seg_labels)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [43]:
ypred=extra_tree_forest.predict(X_seg_flat_test)

In [44]:
accuracy_score(ypred,y_test)

0.38666666666666666

### Grid search cv full features

In [45]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [46]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [47]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3

[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4

[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5

[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] E

[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] E

[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] E

GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [48]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 9}

In [49]:
y_pred1=grid.predict(X_seg_flat_test)

In [50]:
accuracy_score(y_test, y_pred1)

0.5066666666666667

## MLP Classifier

### Basic model Full features

In [51]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [52]:
mlp.fit(X_full_flat_train, y_train)

MLPClassifier(max_iter=500)

In [53]:
prediction=mlp.predict(X_full_flat_test)

In [54]:
accuracy_score(prediction,y_test)

0.3333333333333333

### Grid search cv full features

In [59]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [60]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [61]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  39.2s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  34.8s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.3s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  31.1s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  33.9s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  35.0s
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  29.5s
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  28.3s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  22.3s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 1.6min
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=   5.7s
[CV 2/5] END .....activation=tanh, max_iter=500,

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [62]:
grid.best_params_

{'activation': 'tanh', 'max_iter': 600, 'solver': 'sgd'}

In [63]:
y_pred1=grid.predict(X_full_flat_test)

In [64]:
accuracy_score(y_test, y_pred1)

0.6

### Basic model Full features Oversampling

In [65]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [66]:
mlp.fit(train_full_data, train_full_labels)

MLPClassifier(max_iter=500)

In [67]:
prediction=mlp.predict(X_full_flat_test)

In [68]:
accuracy_score(prediction,y_test)

0.06666666666666667

### Grid search cv full features

In [69]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [70]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [71]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.6s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.9s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  34.7s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.8s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  33.3s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 1.8min
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 2.9min
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  53.1s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  43.7s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 1.5min
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=   7.3s
[CV 2/5] END .....activation=tanh, max_iter=500,

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [72]:
grid.best_params_

{'activation': 'relu', 'max_iter': 500, 'solver': 'lbfgs'}

In [73]:
y_pred1=grid.predict(X_full_flat_test)

In [74]:
accuracy_score(y_test, y_pred1)

0.5733333333333334

### Basic model Segmented features

In [75]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [76]:
mlp.fit(X_seg_flat_train, y_train)

MLPClassifier(max_iter=500)

In [77]:
prediction=mlp.predict(X_seg_flat_test)

In [78]:
accuracy_score(prediction,y_test)

0.5333333333333333

### Grid search cv segmented features

In [79]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [80]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [81]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  12.0s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   3.7s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   5.4s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  21.8s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   9.9s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=   9.2s
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=   9.5s
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  11.9s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.4s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  12.9s
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=   0.5s
[CV 2/5] END .....activation=tanh, max_iter=500,

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [82]:
grid.best_params_

{'activation': 'relu', 'max_iter': 600, 'solver': 'lbfgs'}

In [84]:
y_pred1=grid.predict(X_seg_flat_test)

In [85]:
accuracy_score(y_test, y_pred1)

0.52

### Basic model segmented features Oversampling

In [86]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [87]:
mlp.fit(train_seg_data, train_seg_labels)

MLPClassifier(max_iter=500)

In [88]:
prediction=mlp.predict(X_seg_flat_test)

In [89]:
accuracy_score(prediction,y_test)

0.48

### Grid search cv segmented features Oversampling

In [90]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [91]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [92]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   3.9s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   4.6s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   6.0s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   5.7s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   5.4s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.0s
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  11.2s
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.5s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.4s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=   9.0s
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  14.0s
[CV 2/5] END .....activation=tanh, max_iter=500,

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 3/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  27.9s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  27.9s
[CV 5/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  14.8s
[CV 1/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.6s
[CV 2/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.9s
[CV 3/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   5.2s
[CV 4/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.7s
[CV 5/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.3s
[CV 1/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.2s
[CV 2/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.6s
[CV 3/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.1s
[CV 4/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=  10.4s
[CV 5/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.4s
[CV 1/5] END .....activation

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [93]:
grid.best_params_

{'activation': 'tanh', 'max_iter': 500, 'solver': 'sgd'}

In [95]:
y_pred1=grid.predict(X_seg_flat_test)

In [96]:
accuracy_score(y_test, y_pred1)

0.49333333333333335